Further information I hope to obtain:
* Game descriptions
* Player ratings, alongside other games that those players rated

# Setup

In [1]:
import requests
from lxml import etree
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import sqlite3

In [2]:
con = sqlite3.connect('bgg_full.db')
cur = con.cursor()

In [ ]:
ERROR_MESSAGE = error_message = ['\n\tYour request for this collection has been accepted and will be processed.  Please try again later for access.\n']

In [ ]:
def get_player_count_ratings(game_id):
    params = {'id' : game_id}
    response = requests.get('https://www.boardgamegeek.com/xmlapi2/thing?', params)
    s = BeautifulSoup(response.text)
    player_votings = s.find_all('results')
    player_counter = 0

    # Lookup if a rating refers to best, rec or not rec
    rec_li = ['Best', 'Recommended', 'Not Recommended']


    # output used to make SQL table
    full_rating_li = []

    for val in player_votings:
        try: 
            for idx, elem in enumerate(val.find_all('result')):
                full_rating_li.append((game_id, val['numplayers'], rec_li[idx], elem['numvotes']))
        except KeyError:
            pass

    return full_rating_li 

In [ ]:
def get_user_ratings(username):
    error_message = ['\n\tYour request for this collection has been accepted and will be processed.  Please try again later for access.\n']
    
    got_data = False
    # Make API call to get XML data
    while not got_data:
        params = {'username' : username, 'type': 'boardgame', 'rated' : '1', 'brief' : '1', 'stats' : '1'}
        response = requests.get('https://www.boardgamegeek.com/xmlapi2/collection?', params)
        s = BeautifulSoup(response.text, "xml")
        
        try:
            s.find('message').contents
        except AttributeError:
            got_data = True
        finally:
            time.sleep(1)
    
    # Initialize dataframe
    rating_df = pd.DataFrame(columns = ['id', 'rating'])

    # Loop through and output ratings to a dataframe
    for i in s.find_all('item'):
        game_id = i['objectid']
        
        # Make a soup out of the item info to extract rating
        inner_soup = BeautifulSoup(str(i))
        rv = inner_soup.find('stats')
        inner_soup2 = BeautifulSoup(str(rv))
        rating = inner_soup2.find('rating')
        rating = float(rating['value'])
        
        # Append rating to df
        temp_df = pd.DataFrame([[game_id,rating]], columns = ['id', 'rating'])
        rating_df = rating_df.append(temp_df)
    
    rating_df['index'] = username
    rating_df.rename({'index' :'username'}, inplace=True)
    
    return rating_df

In [ ]:
con = sqlite3.connect('bgg.db')
cur = con.cursor()

# Get Player Counts

In [ ]:
# Get list of top 1000 games from SQL table
game_li = []

results=cur.execute("select id FROM TOP_GAMES_FULL")
results=results.fetchall()

for num in results:
    game_li.append(num[0])

In [ ]:
# For each game, get the min and max player count:
min_max_player_li = []

i = 0
while i < len(game_li):
    game_id = game_li[i]
    params = {'id' : game_id,'stats' :'1'}
    response = requests.get('https://www.boardgamegeek.com/xmlapi2/thing?', params)
    soup = BeautifulSoup(response.text, "xml")

    try: 
        min_play = soup.find('minplayers')['value']
        max_play = soup.find('maxplayers')['value']
        i += 1
    except TypeError:
        # If it fails, we need to wait so we can use the API gain
        time.sleep(15)
    
    min_max_player_li.append([game_id, min_play, max_play])

In [ ]:
# Create table
cur.execute(''' CREATE TABLE BGG_PLAYER_COUNTS
               (id text, min_players text, max_players text)''')

In [ ]:
# Remove duplicates from API
player_li_df = pd.DataFrame(min_max_player_li, columns = ['GameId','MinPlayers','MaxPlayers'])
player_li_df.drop_duplicates(subset = 'GameId', keep='first', inplace=True)
player_li_df.reset_index(inplace=True)
player_li_df.shape

In [ ]:
# Convert dataframe to tuple
df = player_li_df
tuple_li = []
for i in range(len(df)):
    tuple_ = str(df.loc[i, 'GameId']), df.loc[i, 'MinPlayers'], str(df.loc[i, 'MaxPlayers'])
    tuple_li.append(tuple_)

In [ ]:
# Enter tuple into SQL table
cur.executemany("insert into BGG_PLAYER_COUNTS values (?, ?, ?)", tuple_li)

In [ ]:
con.commit()

# Obtain game categories and mechanism

In [ ]:
category_li, mechanic_li = [],[]
loop_len = len(game_li)

i = 0
while i < loop_len:
    game_id = game_li[i]
    params = {'id' : game_id}
    response = requests.get('https://www.boardgamegeek.com/xmlapi2/thing?', params)
    soup = BeautifulSoup(response.text, 'xml')
    info_ = soup.find_all('link')
    
    try:
        if info_ == []:
            raise TypeError
        for x in info_:
            if x['type'] == 'boardgamecategory':
                category_li.append((game_id, x['value']))
            elif x['type'] == 'boardgamemechanic':
                mechanic_li.append((game_id, x['value']))
        i += 1 # increment the loop
        print(i)
    except TypeError:  # occurs when API times out
        time.sleep(10) # wait ten seconds so we can run the API again

In [ ]:
li__ = [elem[0] for elem in mechanic_li]

In [ ]:
# Create table
cur.execute(''' CREATE TABLE BGG_CATEGORIES
               (id text, category text);
               ''')

cur.execute('''CREATE TABLE BGG_MECHANICS
               (id text, mechanic text);''')

In [ ]:
# Enter tuple into SQL table
cur.executemany("insert into BGG_CATEGORIES values (?, ?)", category_li)
cur.executemany("insert into BGG_MECHANICS values (?, ?)", mechanic_li)

In [ ]:
con.commit()

# Key Game Stats

In [ ]:
def get_game_stats(game):
    params = {'id' : game,'stats' :'1'}
    response = requests.get('https://www.boardgamegeek.com/xmlapi2/thing?', params)
    s = BeautifulSoup(response.text, "xml")
    all_stats = s.find('ratings')
    avg_rating = all_stats.find('average')['value']
    avg_bayes_rating = all_stats.find('bayesaverage')['value']
    std = all_stats.find('stddev')['value']
    ratings = all_stats.find('usersrated')['value']
    weight = all_stats.find('averageweight')['value']
    
    return((game, avg_rating, avg_bayes_rating, std, ratings, weight))

In [ ]:
game_rating_li = []
for idx, game in enumerate(game_li):
    got_data = False
    
    while not got_data:
        try:
            temp = get_game_stats(game)
            got_data = True
        except AttributeError:
            time.sleep(10)
    # print(idx) 
    game_rating_li.append(temp)

In [ ]:
cur.execute('''CREATE TABLE BGG_GAME_STATS
               (id text, avg_rating text, avg_bayes_rating text, std text, ratings text, weight text);''')

In [ ]:
cur.executemany("insert into BGG_GAME_STATS values (?, ?, ?, ?, ?, ?)", game_rating_li)

In [ ]:
con.commit()

# Num of Players Voting

In [ ]:
full_player_rating_li = []
for g in game_li:
    current_li = get_player_count_ratings(g)
    if current_li == []:
        time.sleep(10)
        current_li = get_player_count_ratings(g)
    full_player_rating_li.extend(current_li)

In [ ]:
cur.execute(''' CREATE TABLE BGG_PLAYER_RATINGS
               (id text, player_count text, player_recommendation text, votes text);
               ''')

In [ ]:
cur.executemany("insert into BGG_PLAYER_RATINGS values (?, ?, ?, ?)", full_player_rating_li)

In [ ]:
con.commit()

# Join Tables

In [ ]:
cur.execute('''CREATE TABLE TOP_GAMES_FULL
           
           AS
           
        SELECT t1.id, title, year, t2.min_players, t2.max_players
           ,avg_rating, avg_bayes_rating, std, ratings, weight
        FROM TOP_BGG_GAMES t1
            LEFT JOIN BGG_PLAYER_COUNTS t2
                ON t1.id = t2.id
            LEFT JOIN  BGG_GAME_STATS t3
                ON t1.id = t3.id
           
           ''')

In [ ]:
con.commit()

In [ ]:
results = cur.execute('''
        SELECT t1.id, title, max_players
           ,avg_rating, std
        FROM TOP_GAMES_FULL t1
           
           ''')

In [ ]:
result_li = [result for result in results]

In [ ]:
df = pd.DataFrame(data=result_li, columns=['id', 'title', 'max_players', 'score', 'std_score'])

In [ ]:
# convert data types
cols = ['max_players', 'score', 'std_score']
for col in cols:
    df[col] = df[col].astype('float')
df.dtypes

In [ ]:
# solo games 
df[df.max_players == 1]

In [ ]:
# duel games
df[df.max_players == 2]

In [ ]:
# Large Group games
df[df.max_players > 5]

In [ ]:
#What's the largest?
print(df.max_players.max())

# Which games?
df[df['max_players'] == 100]

In [ ]:
# What's the most divisive game?
df[df['std_score']==df.std_score.max()]

In [ ]:
# The least divisive?
df[df['std_score']==df.std_score.min()]

In [ ]:
params = {'id' : 13,'stats' :'1'}
response = requests.get('https://www.boardgamegeek.com/xmlapi2/thing?', params)
s = BeautifulSoup(response.text, "xml")
s.find('playingtime')

# Coda: Adding Play time

Initially, I felt that the previous table was complete. I decided to add playtime as well. I should have added it before, though this gives me the chance to test the Alter table functions

In [ ]:
i = 0
playtime_li = []
while i < len(game_li):
    game_id = game_li[i]
    params = {'id' : game_id,'stats' :'1'}
    try:
        response = requests.get('https://www.boardgamegeek.com/xmlapi2/thing?', params)
        s = BeautifulSoup(response.text, "xml")
        play_time = s.find('playingtime')['value']
        playtime_li.append((game_id, play_time))
        i += 1
    except TypeError:
        time.sleep(20)

In [ ]:
cur.execute(''' CREATE TABLE BGG_PLAY_TIME
               (id text, play_time text);
               ''')

cur.executemany("insert into BGG_PLAY_TIME values (?, ?)", playtime_li)

In [ ]:
cur.execute('''  ALTER TABLE TOP_GAMES_FULL
     ADD COLUMNS play_time text ''')

In [ ]:
cur.execute(''' DELETE  FROM TOP_GAMES_FULL  WHERE TRUE''')

In [ ]:
cur.execute('''INSERT INTO TOP_GAMES_FULL
           
        SELECT t1.id, title, year, t2.min_players, t2.max_players
           ,avg_rating, avg_bayes_rating, std, ratings, weight, t4.play_time
        FROM TOP_BGG_GAMES t1
            LEFT JOIN BGG_PLAYER_COUNTS t2
                ON t1.id = t2.id
            LEFT JOIN  BGG_GAME_STATS t3
                ON t1.id = t3.id
            LEFT JOIN BGG_PLAY_TIME t4
                ON t1.id = t4.id''')

In [ ]:
results = cur.execute('PRAGMA table_info(TOP_GAMES_FULL);')
results = results.fetchall()
result_li = [r for r in results]
result_li

In [ ]:
cur.execute('ALTER TABLE TOP_GAMES_FULL RENAME COLUMN COLUMNS TO play_time;')

In [ ]:
con.commit()

Playtime added!

# Load Game Ratings

In [4]:
results=cur.execute('''SELECT DISTINCT game_id 
                    
                    FROM BGG_GAME_RATINGS
                    
                    ORDER BY 1 ASC
                    ''')
results=results.fetchall()
last_result= results[len(results) - 1]
last_result = last_result[0]
print(last_result)

164265


In [5]:
cur.execute('''Delete from BGG_GAME_RATINGS WHERE game_id >= {0} '''.format(last_result))

In [6]:
# Get list of top 1000 games from SQL table
game_li = []

results=cur.execute('''select id FROM TOP_GAMES_FULL
                    
                    WHERE 1=1
                        AND id >= {0}
                    
                    ORDER BY 1 ASC
                    '''.format(last_result))
results=results.fetchall()

for num in results:
    game_li.append(num[0])
    game_li.sort()

In [7]:
print(len(game_li))

748


In [8]:
# cur.execute(''' CREATE TABLE BGG_GAME_RATINGS
#                (game_id text, username text, rating);
#                ''')

In [ ]:
con.commit()

In [ ]:
# game_id = game_li[i]
for game in game_li:
    i = 1
    comment_li = []
    while True:
        params = {'id' : game, 'ratingcomments': '1', 'page' : i, 'pagesize':100}
        response = requests.get('https://www.boardgamegeek.com/xmlapi2/thing?', params)
        soup = BeautifulSoup(response.text, "xml")
        comments = soup.find_all("comment")
        if len(comments) == 0:
            break
        else:
            pass # print(i, ':', len(comments))
        if i % 10 == 0:
            print('game:', game, 'page:', i)
            time.sleep(30)
        for com in comments:
            temp_tuple = (game, com['username'], com['rating'])
            comment_li.append(temp_tuple)
        i += 1
    print('completed coments for game:', game)
    cur.executemany("insert into BGG_GAME_RATINGS values (?, ?, ?)", comment_li)
    con.commit()

game: 164265 page: 10
game: 164265 page: 20
completed coments for game: 164265
game: 164928 page: 10
game: 164928 page: 20
game: 164928 page: 30
game: 164928 page: 40
game: 164928 page: 50
game: 164928 page: 60
game: 164928 page: 70
game: 164928 page: 80
game: 164928 page: 90
game: 164928 page: 100
game: 164928 page: 110
game: 164928 page: 120
game: 164928 page: 130
game: 164928 page: 140
game: 164928 page: 150
game: 164928 page: 160
game: 164928 page: 170
game: 164928 page: 180
game: 164928 page: 190
game: 164928 page: 200
game: 164928 page: 210
game: 164928 page: 220
game: 164928 page: 230
game: 164928 page: 240
completed coments for game: 164928
game: 164949 page: 10
completed coments for game: 164949
game: 165401 page: 10
game: 165401 page: 20
completed coments for game: 165401
game: 165722 page: 10
game: 165722 page: 20
game: 165722 page: 30
game: 165722 page: 40
game: 165722 page: 50
game: 165722 page: 60
completed coments for game: 165722
game: 165838 page: 10
completed coments 

game: 170216 page: 90
game: 170216 page: 100
game: 170216 page: 110
game: 170216 page: 120
game: 170216 page: 130
game: 170216 page: 140
game: 170216 page: 150
game: 170216 page: 160
game: 170216 page: 170
game: 170216 page: 180
game: 170216 page: 190
game: 170216 page: 200
game: 170216 page: 210
game: 170216 page: 220
game: 170216 page: 230
game: 170216 page: 240
game: 170216 page: 250
game: 170216 page: 260
game: 170216 page: 270
game: 170216 page: 280
game: 170216 page: 290
game: 170216 page: 300
game: 170216 page: 310
game: 170216 page: 320
game: 170216 page: 330
game: 170216 page: 340
game: 170216 page: 350
game: 170216 page: 360
completed coments for game: 170216
game: 170416 page: 10
game: 170416 page: 20
game: 170416 page: 30
game: 170416 page: 40
game: 170416 page: 50
game: 170416 page: 60
completed coments for game: 170416
game: 170561 page: 10
game: 170561 page: 20
game: 170561 page: 30
game: 170561 page: 40
completed coments for game: 170561
game: 170624 page: 10
game: 1706

completed coments for game: 173442
game: 17392 page: 10
game: 17392 page: 20
game: 17392 page: 30
completed coments for game: 17392
game: 17396 page: 10
game: 17396 page: 20
completed coments for game: 17396
game: 17405 page: 10
game: 17405 page: 20
completed coments for game: 17405
game: 174430 page: 10
game: 174430 page: 20
game: 174430 page: 30
game: 174430 page: 40
game: 174430 page: 50
game: 174430 page: 60
game: 174430 page: 70
game: 174430 page: 80
game: 174430 page: 90
game: 174430 page: 100
game: 174430 page: 110
game: 174430 page: 120
game: 174430 page: 130
game: 174430 page: 140
game: 174430 page: 150
game: 174430 page: 160
game: 174430 page: 170
game: 174430 page: 180
game: 174430 page: 190
game: 174430 page: 200
game: 174430 page: 210
game: 174430 page: 220
game: 174430 page: 230
game: 174430 page: 240
game: 174430 page: 250
game: 174430 page: 260
game: 174430 page: 270
game: 174430 page: 280
game: 174430 page: 290
game: 174430 page: 300
game: 174430 page: 310
game: 174430

game: 18 page: 170
game: 18 page: 180
game: 18 page: 190
game: 18 page: 200
game: 18 page: 210
game: 18 page: 220
game: 18 page: 230
completed coments for game: 18
game: 180263 page: 10
game: 180263 page: 20
game: 180263 page: 30
game: 180263 page: 40
game: 180263 page: 50
game: 180263 page: 60
game: 180263 page: 70
game: 180263 page: 80
game: 180263 page: 90
game: 180263 page: 100
game: 180263 page: 110
game: 180263 page: 120
game: 180263 page: 130
game: 180263 page: 140
game: 180263 page: 150
game: 180263 page: 160
game: 180263 page: 170
game: 180263 page: 180
completed coments for game: 180263
game: 180511 page: 10
game: 180511 page: 20
game: 180511 page: 30
game: 180511 page: 40
completed coments for game: 180511
game: 180593 page: 10
game: 180593 page: 20
game: 180593 page: 30
game: 180593 page: 40
game: 180593 page: 50
game: 180593 page: 60
completed coments for game: 180593
game: 180680 page: 10
game: 180680 page: 20
game: 180680 page: 30
completed coments for game: 180680
game:

# Scratch Code

In [ ]:
# We look at certain users from the comments and see what other games they might like
params = {'name' : 'agentzen', 'hot' : '1', 'top' : '1', 'domain' : 'boardgame',}
response = requests.get('https://www.boardgamegeek.com/xmlapi2/user?', params)
s = BeautifulSoup(response.text, "xml")
s

In [ ]:
# We look at certain users from the comments and see what other games they might like
params = {'username' : 'agentzen', 'type': 'boardgame', 'rated' : '1', 'brief' : '1', 'stats' : '1'}
response = requests.get('https://www.boardgamegeek.com/xmlapi2/collection?', params)
s = BeautifulSoup(response.text, "xml")
s

In [ ]:
# Comments
comment_li = []

# Obtaining ten pages of user comments/ratings
for i in range(1,11):
    params = {'id' : '13', 'ratingcomments' : '1', 'comments' : '1', 'pagesize' : '100', 'page':[3,4], 'stats' :str(i)}
    response = requests.get('https://www.boardgamegeek.com/xmlapi2/thing?', params)
    comments = soup.find_all("comment")
    for com in comments:
        comment_li.append(com)

In [ ]:
test_df = get_user_ratings('agent_t_bib')
test_df